
* Under refactoring. See `pycalib/ba.py`.


In [3]:
%matplotlib notebook
import sys, os, cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pycalib.plot import plotCamera
from pycalib.ba import Projection, Camera


In [4]:
# 3D points
# X_gt = (np.random.rand(16, 3) - 0.5)*5 # random points centered at [0, 0, 0]
X_gt = np.array(np.meshgrid(np.linspace(-1, 1, 3), np.linspace(-1, 1, 3), np.linspace(-1, 1, 3))).reshape((3, -1)).T  # 3D grid points
Np = X_gt.shape[0]
print('X_gt:', X_gt.shape)

# Camera intrinsics
K = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]]).astype(np.float)  # VGA camera

# Camera poses: cameras are at the vertices of a hexagon
t = 2 * np.pi / 5 * np.arange(5)
v_gt = np.vstack((10*np.cos(t), 10*np.sin(t), np.zeros(t.shape))).T
Nc = v_gt.shape[0]
R_gt = []
t_gt = []
P_gt = []
rvec_gt = []
for i in range(Nc):
    t = v_gt[i,:]
    R, t = lookat(t, np.zeros(3), np.array([0, 1, 0]))
    R_gt.append(R)
    t_gt.append(t)
    P_gt.append(K @ np.hstack((R, t)))
    rvec_gt.append(cv2.Rodrigues(R)[0])
R_gt = np.array(R_gt)
t_gt = np.array(t_gt)
P_gt = np.array(P_gt)
rvec_gt = np.array(rvec_gt)
print('R_gt:', R_gt.shape)
print('t_gt:', t_gt.shape)
print('P_gt:', P_gt.shape)
print('rvec_gt:', rvec_gt.shape)

# 2D observations points
x = []
for i in range(Nc):
    xt = cv2.projectPoints(X_gt.reshape((-1, 1, 3)), rvec_gt[i], t_gt[i], K, None)[0].reshape((-1, 2))
    x.append(xt)
x = np.array(x)
print('x:', x.shape)

# Verify triangulation
Y = []
for i in range(Np):
    y = triangulate(x[:,i,:].reshape((-1,2)), P_gt)
    #print(y)
    Y.append(y)
Y = np.array(Y).T
Y = Y[:3,:] / Y[3,:]
assert np.allclose(0, X_gt - Y.T)

# Verify z > 0 at each camera
for i in range(Nc):
    Xc = R_gt[i] @ X_gt.T + t_gt[i]
    assert np.all(Xc[2, :] > 0)


X_gt: (27, 3)
R_gt: (5, 3, 3)
t_gt: (5, 3, 1)
P_gt: (5, 3, 4)
rvec_gt: (5, 3, 1)
x: (5, 27, 2)


add gaussian noise to the ground truth

In [7]:
R_est = R_gt.copy()
t_est = R_gt.copy()
K_est = np.array([K for c in range(Nc)])

[[[600.   0. 320.]
  [  0. 600. 240.]
  [  0.   0.   1.]]

 [[600.   0. 320.]
  [  0. 600. 240.]
  [  0.   0.   1.]]

 [[600.   0. 320.]
  [  0. 600. 240.]
  [  0.   0.   1.]]

 [[600.   0. 320.]
  [  0. 600. 240.]
  [  0.   0.   1.]]

 [[600.   0. 320.]
  [  0. 600. 240.]
  [  0.   0.   1.]]]


Bundle adjustment

In [9]:
# build the model
cams = []
for i in range(Nc):
    cam = Camera(cv2.Rodrigues(R_est[i])[0], t_est[i], K_est[i][0,0], None, K_est[i][0,2], K_est[i][1,2], np.zeros(5))
    cams.append(cam)

    
########

masks = torch.from_numpy(mask)
pt2ds = torch.from_numpy(-observations[:, 2:])
assert masks.shape == (num_cameras, num_points)

model = Projection(cams, points.T)
print(model)

device = torch.device('cpu')

model = model.to(device)
masks = masks.to(device)
pt2ds = pt2ds.to(device)

optimizer = optim.Adadelta(model.parameters(), lr=1e-2)
criterion = torch.nn.MSELoss()

model.train()
for i in range(1000):
    # 順伝搬
    x = model.forward(masks)
    # 再投影誤差を計算
    loss = criterion(x, pt2ds)
    print('E_rep[%d] = %f px (mse)' % (i, loss))
    #print(x[0, :], pt2ds[0, :])
    # 勾配をゼロ初期化
    optimizer.zero_grad()
    # 勾配の計算
    loss.backward()
    # パラメータの更新
    optimizer.step()

print(model.cpu())

NameError: name 'torch' is not defined